# Preprocessing

In [1]:
%pip install pydriller --upgrade

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pydriller import Repository, Git
import os
import json

In [3]:
# Clone code from the repo and save it for code portability -> via normal git clone
url = "https://github.com/apache/kafka"
repo_path = os.path.join(os.getcwd(), 'kafka')
clone = f"git clone {url} \"{repo_path}\""

os.system(clone) # Cloning

128

In [4]:
# checkout the tag 3.6.0
os.chdir(repo_path)
os.system("git checkout 3.6.0")
# back to the "home" folder
os.chdir("..")

# Exercise 1

In [5]:
# Configuration
path = "./kafka"
save_path = "./ex_1_result.json"
tag = "3.6.0"
from_ = "3.5.1"

In [ ]:
# List all entities (files) of tag
gr = Git(path)
commit = gr.get_commit_from_tag(tag)
gr.checkout(commit.hash)
file_list = gr.files()

In [15]:
#Get the number of revisions and authors of each entity in a given list.
result = {
    file.split("\\")[-1]: {"num of revisions": 0, "authors": {}} for file in file_list
}
from_commit_date = gr.get_commit_from_tag(from_).committer_date
to_commit_date = gr.get_commit_from_tag(tag).committer_date

for commit in Repository(path, since=from_commit_date, to=to_commit_date).traverse_commits():
    for file in commit.modified_files:
        filename = file.filename
        if filename not in result:
            continue
        result[filename]["num of revisions"] += 1
        if commit.author.name not in result[filename]["authors"]:
            result[filename]["authors"][commit.author.name] = {
                "added": commit.insertions,
                "deleted": commit.deletions,
                "total": commit.lines,
            }
        else:
            result[filename]["authors"][commit.author.name]["added"] += commit.insertions
            result[filename]["authors"][commit.author.name]["deleted"] += commit.deletions
            result[filename]["authors"][commit.author.name]["total"] += commit.lines

In [ ]:
#Save the result as a JSON file.
with open(save_path, "w") as f:
    json.dump(result, f, indent=4)

print("Saved as {}".format(save_path))